In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L2\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['COAST.csv', 'EAST.csv', 'FWEST.csv', 'NCENT.csv', 'NORTH.csv', 'SCENT.csv', 'SOUTH.csv', 'WEST.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[3]))

,Net,Temprature
0,13640.024978,4.6
1,13425.121941,4.6
2,13286.070909,4.5
3,13310.892590,4.3
4,13423.345503,3.9
...,...,...
43818,13164.660616,13.4
43819,12928.911509,13.8
43820,12716.274141,13.7
43821,12413.090126,12.5


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[3]

zoneData = pd.read_csv(os.path.join(path, fileName))
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_NCENT,Temp_NCENT
0,13640.024978,4.6
1,13640.024978,4.6
2,13425.121941,4.6
3,13286.070909,4.5
4,13310.892590,4.3
...,...,...
43819,13164.660616,13.4
43820,12928.911509,13.8
43821,12716.274141,13.7
43822,12413.090126,12.5


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_NCENT,Temp_NCENT,Lag-2,Lag-7
0,17906.589200,4.6,13710.913700,13640.024978
1,17453.598477,3.0,13179.594044,13640.024978
2,17277.881196,1.6,12956.587981,13425.121941
3,17309.339711,0.2,12929.643478,13286.070909
4,17505.010490,-1.3,13128.760126,13310.892590
...,...,...,...,...
43651,13164.660616,13.4,13575.920065,11434.247698
43652,12928.911509,13.8,13552.882422,11231.263595
43653,12716.274141,13.7,13453.520609,11031.084524
43654,12413.090126,12.5,12987.897930,10749.333909


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 13s 8ms/step - loss: 0.0125 - mape: 3469.6304 - mae: 0.0837 - val_loss: 0.0061 - val_mape: 28.7888 - val_mae: 0.0614
Epoch 2/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0076 - mape: 2211.6091 - mae: 0.0662 - val_loss: 0.0148 - val_mape: 33.5929 - val_mae: 0.0887
Epoch 3/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0054 - mape: 1975.3279 - mae: 0.0555 - val_loss: 0.0210 - val_mape: 44.2188 - val_mae: 0.1059
Epoch 4/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0048 - mape: 1730.7618 - mae: 0.0522 - val_loss: 0.0181 - val_mape: 37.8049 - val_mae: 0.0984
Epoch 5/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0044 - mape: 1566.1018 - mae: 0.0501 - val_loss: 0.0221 - val_mape: 41.6581 - val_mae: 0.1068
Epoch 6/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0042 - mape: 1734.4047 - mae: 0.0491 - val_loss: 0.0209

1105/1105 [==============================] - 8s 7ms/step - loss: 0.0021 - mape: 1033.2649 - mae: 0.0334 - val_loss: 0.0195 - val_mape: 35.0183 - val_mae: 0.1020
Epoch 49/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0020 - mape: 911.9404 - mae: 0.0331 - val_loss: 0.0193 - val_mape: 35.5924 - val_mae: 0.1013
Epoch 50/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0020 - mape: 716.3168 - mae: 0.0327 - val_loss: 0.0183 - val_mape: 33.2017 - val_mae: 0.0960
Epoch 51/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0020 - mape: 1051.3369 - mae: 0.0328 - val_loss: 0.0207 - val_mape: 35.7959 - val_mae: 0.1046
Epoch 52/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0019 - mape: 922.1035 - mae: 0.0324 - val_loss: 0.0192 - val_mape: 33.6630 - val_mae: 0.0990
Epoch 53/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0019 - mape: 862.4594 - mae: 0.0323 - val_loss: 0.0210 - val_mape:

1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 765.3469 - mae: 0.0247 - val_loss: 0.0184 - val_mape: 32.4554 - val_mae: 0.0968
Epoch 96/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 651.0001 - mae: 0.0247 - val_loss: 0.0201 - val_mape: 34.9174 - val_mae: 0.1028
Epoch 97/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 962.5476 - mae: 0.0244 - val_loss: 0.0197 - val_mape: 33.6984 - val_mae: 0.0997
Epoch 98/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 1019.8465 - mae: 0.0243 - val_loss: 0.0193 - val_mape: 33.4347 - val_mae: 0.0993
Epoch 99/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0010 - mape: 603.0565 - mae: 0.0241 - val_loss: 0.0193 - val_mape: 33.3331 - val_mae: 0.0996
Epoch 100/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 943.8434 - mae: 0.0247 - val_loss: 0.0201 - val_mape:

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 835.0351634025574


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 8ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.09141159, 0.06661786, 0.06734449, ..., 0.35445017, 0.319644  ,
       0.2926563 ], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.28139508, 0.24340256, 0.22392499, ..., 0.28336127, 0.26759737,
       0.25052683])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,12678.458738,9024.544922
1,11947.756347,8547.692383
2,11573.148128,8561.666992
3,11526.586317,8767.881836
4,11893.973828,9026.034180
...,...,...
4363,13164.660616,15071.098633
4364,12928.911509,14702.140625
4365,12716.274141,14083.512695
4366,12413.090126,13414.092773


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

12.92276661787492